# Mood Detection

In [3]:
!pip install essentia-tensorflow

In [2]:
# Create directories
!mkdir -p models/mood_detection_models
!mkdir -p metadata/mood_detection_metadatas

# Download .pb file
!wget -O models/mood_detection_models/danceability-musicnn-msd-1.pb https://essentia.upf.edu/models/classifiers/danceability/danceability-musicnn-msd-1.pb

# Download .json file
!wget -O metadata/mood_detection_metadatas/danceability-musicnn-msd-1.json https://essentia.upf.edu/models/classifiers/danceability/danceability-musicnn-msd-1.json

# Happy Mood Models
!wget -O models/mood_detection_models/mood_happy-musicnn-msd-1.pb https://essentia.upf.edu/models/classifiers/mood_happy/mood_happy-musicnn-msd-1.pb
!wget -O metadata/mood_detection_metadatas/mood_happy-musicnn-msd-1.json https://essentia.upf.edu/models/classifiers/mood_happy/mood_happy-musicnn-msd-1.json

# Party Mood Models
!wget -O models/mood_detection_models/mood_party-musicnn-msd-1.pb https://essentia.upf.edu/models/classifiers/mood_party/mood_party-musicnn-msd-1.pb
!wget -O metadata/mood_detection_metadatas/mood_party-musicnn-msd-1.json https://essentia.upf.edu/models/classifiers/mood_party/mood_party-musicnn-msd-1.json

# Sad Mood Models
!wget -O models/mood_detection_models/mood_sad-musicnn-msd-1.pb https://essentia.upf.edu/models/classifiers/mood_sad/mood_sad-musicnn-msd-1.pb
!wget -O metadata/mood_detection_metadatas/mood_sad-musicnn-msd-1.json https://essentia.upf.edu/models/classifiers/mood_sad/mood_sad-musicnn-msd-1.json

# Aggressive Mood Models
!wget -O models/mood_detection_models/mood_aggressive-musicnn-msd-1.pb https://essentia.upf.edu/models/classifiers/mood_aggressive/mood_aggressive-musicnn-msd-1.pb
!wget -O metadata/mood_detection_metadatas/mood_aggressive-musicnn-msd-1.json https://essentia.upf.edu/models/classifiers/mood_aggressive/mood_aggressive-musicnn-msd-1.json

# Electronic Mood Models
!wget -O models/mood_detection_models/mood_electronic-musicnn-msd-1.pb https://essentia.upf.edu/models/classifiers/mood_electronic/mood_electronic-musicnn-msd-1.pb
!wget -O metadata/mood_detection_metadatas/mood_electronic-musicnn-msd-1.json https://essentia.upf.edu/models/classifiers/mood_electronic/mood_electronic-musicnn-msd-1.json

# Relaxed Mood Models
!wget -O models/mood_detection_models/mood_relaxed-musicnn-msd-1.pb https://essentia.upf.edu/models/classifiers/mood_relaxed/mood_relaxed-musicnn-msd-1.pb
!wget -O metadata/mood_detection_metadatas/mood_relaxed-musicnn-msd-1.json https://essentia.upf.edu/models/classifiers/mood_relaxed/mood_relaxed-musicnn-msd-1.json

--2024-12-13 18:10:44--  https://essentia.upf.edu/models/classifiers/danceability/danceability-musicnn-msd-1.pb
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3239625 (3.1M) [application/octet-stream]
Saving to: ‘models/mood_detection_models/danceability-musicnn-msd-1.pb’

models/mood_detecti 100%[===================>]   3.09M   528KB/s    in 6.6s    

2024-12-13 18:10:52 (476 KB/s) - ‘models/mood_detection_models/danceability-musicnn-msd-1.pb’ saved [3239625/3239625]

--2024-12-13 18:10:52--  https://essentia.upf.edu/models/classifiers/danceability/danceability-musicnn-msd-1.json
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2677 (2.6K) [application/json]
Saving to: ‘metadata/mood_dete

In [4]:
# import os
# import json
from essentia.standard import MonoLoader, TensorflowPredictMusiCNN

class AudioMoodClassifier:
    def __init__(self, models_dir,metadatas_dir):
        """
        Initialize the MoodClassifier with mood models directory and metadatas dir

        Args:
            models_dir (str): Directory containing model .pb and .json files
        """
        self.models_dir = models_dir
        self.metadatas_dir = metadatas_dir
        self.models = []
        self.models_names = []
        self.metadatas = []
        self._load_models()

    def _load_models(self):
        """
        Load models and their respective json files.
        """
        # Find all .pb and .json files
        pb_files = [f for f in os.listdir(self.models_dir) if f.endswith('.pb')]
        json_files = [f for f in os.listdir(self.metadatas_dir) if f.endswith('.json')]

        # Sort files to ensure matching .pb and .json files
        pb_files.sort()
        json_files.sort()

        print(pb_files)
        print(json_files)

        # Load metadata and models
        for pb_file, json_file in zip(pb_files, json_files):
            # Full paths
            pb_path = os.path.join(self.models_dir, pb_file)
            json_path = os.path.join(self.metadatas_dir, json_file)

            # Load metadata
            print(f'JSON PATH: {json_path}')
            with open(json_path, 'r') as f:
                metadata = json.load(f)

            # Prepare model
            model = TensorflowPredictMusiCNN(graphFilename=pb_path)
            name = pb_file.replace('.pb','')

            # Store model and metadata and names
            self.models_names.append(name)
            self.models.append(model)
            self.metadatas.append(metadata)

    def predict(self, audio_path, sample_rate=16000):
        """
        Predict audio Mood Classification.

        Args:
            audio_path (str): Path to the audio file
            sample_rate (int): Sampling rate for audio loading

        Returns:
            dict: Classification probabilities for each model
        """
        # Load audio
        loader = MonoLoader(sampleRate=sample_rate, filename=audio_path)
        audio = loader()

        # Predictions from all models.
        results = {}

        for metadata, model, model_name in zip(self.metadatas, self.models,self.models_names):
            # Get model name from the metadata filename
            model_name = model_name.replace('-musicnn-msd-1', '')

            # Compute model activations and take mean across time
            activations = model(audio)
            mean_activations = activations.mean(axis=0)

            for label, probability in zip(metadata['classes'], mean_activations):
              if not label.startswith('non') and not label.startswith('not'):
                results[label] = float(f'{100 * probability:.1f}')

        #Get Top 3 Predictions.
        results = self.get_top_3_predictions(results)

        #Return Top 3 Moods Of the Song.
        return results

    def get_top_3_predictions(self, predictions):
        """
        Get top 3 predictions from a prediction dictionary

        Args:
            predictions (dict): Predictions dictionary

        Returns:
            list: Top 3 predictions as [(label, probability)]
        """
        # Sort predictions by probability in descending order
        return sorted(predictions.items(), key=lambda x: x[1], reverse=True)[:3]

    def print_predictions(self, predictions):
        """
        Print predictions in a formatted manner.

        Args:
            predictions (dict): Prediction results from predict method
        """
        for model_name, classes in predictions.items():
            print(f"\n{model_name} Predictions:")
            for label, probability in classes.items():
                print(f'{label}: {probability}%')

ImportError: cannot import name 'TensorflowPredictMusiCNN' from 'essentia.standard' (/opt/anaconda3/envs/music_analysis_env_2/lib/python3.10/site-packages/essentia/standard.py)

# KEY AND BPM DETECTOR

In [4]:
import essentia
import numpy as np
import essentia.standard as estd
from essentia.standard import MonoLoader, KeyExtractor
import librosa

class KeyBPMExtractor:
    def __init__(self):
        """
        Initialize the KeyBPMExtractor with a specific audio file.
        """
        self.audio = None
        self.sr = None

    def _load_audio(self,file_path):
        """
        Load audio using Essentia's MonoLoader and librosa.
        """
        try:
            self.file_path = file_path
            #Feature Extraction for tonal based Approaach of extraction of key/BPM from audio.
            self.features, self.features_frames = estd.MusicExtractor(lowlevelStats=['mean', 'stdev'],
                                                          rhythmStats=['mean', 'stdev'],
                                                          tonalStats=['mean', 'stdev'])(self.file_path)

            # Load with librosa to get sample rate for BPM Detection.
            self.y, self.sr = librosa.load(self.file_path, sr=None)

        except Exception as e:
            print(f"Error loading audio file {self.file_path}: {e}")
            raise

    def extract_key(self):
        """
        Extract the musical key of the audio file.

        Returns:
            str: Formatted key and scale (e.g., "C Major")
        """
        try:

            # key_extractor = KeyExtractor()
            # key, scale, _ = key_extractor(self.audio)

            keys,scales = [],[]

            #Get Key from tonal.key_edma.key for analysis.
            keys.append(self.features['tonal.key_edma.key'])
            scales.append(self.features['tonal.key_edma.scale'])

            #Get Key from tonal.key_krumhansl.key for analysis.
            keys.append(self.features['tonal.key_krumhansl.key'])
            scales.append(self.features['tonal.key_krumhansl.scale'])

            #Get Key from tonal.key_temperley.key for analysis.
            keys.append(self.features['tonal.key_temperley.key'])
            scales.append(self.features['tonal.key_temperley.scale'])

            #Get Key from librosa.
            chroma = librosa.feature.chroma_cqt(y=self.y, sr=self.sr)
            chroma_mean = np.mean(chroma,axis=1)
            lib_keys = [
                "C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"
            ]
            key_index = np.argmax(chroma_mean)
            predicted_key = lib_keys[key_index]

            detection_key_scales = [f"{key} {scale}" for key, scale in zip(keys, scales)]

            detection_key_scales.append(predicted_key)
            print(f'Detected Key Scales are: {detection_key_scales}')

            return detection_key_scales

        except Exception as e:
            print(f"Error extracting key for {self.file_path}: {e}")
            return "Key Not Found"

    def extract_bpm(self):
        """
        Extract the tempo (BPM) of the audio file.

        Returns:
            int: Rounded beats per minute
        """
        try:
            bpms = []
            #Extracting BPM using Essentia.
            bpm = self.features['rhythm.bpm']
            bpms.append(round(float(bpm)))

            #Extracting BPM using librosa.beat_track.
            tempo, _ = librosa.beat.beat_track(y=self.y, sr=self.sr)
            bpms.append(round(float(tempo)))

            #Using librosa.beat.tempo for analysis now.
            tempo = librosa.beat.tempo(y=self.y, sr=self.sr)[0]

            bpms.append(round(float(tempo)))

            return bpms
        except Exception as e:
            print(f"Error extracting BPM for {self.file_path}: {e}")
            return 0

    def analyze(self,file_path):
        """
        Perform complete audio analysis.

        Returns:
            dict: Dictionary containing analysis results
        """
        self._load_audio(file_path)

        results = {
            'Key': self.extract_key(),
            'BPM' : self.extract_bpm()
        }

        return results


# Instrument Detection

In [5]:
# Download the model file into the models directory
!wget -q https://essentia.upf.edu/models/classification-heads/mtg_jamendo_instrument/mtg_jamendo_instrument-discogs-effnet-1.pb -O models/mtg_jamendo_instrument-discogs-effnet-1.pb

# Download the metadata file into the metadata directory
!wget -q https://essentia.upf.edu/models/classification-heads/mtg_jamendo_instrument/mtg_jamendo_instrument-discogs-effnet-1.json -O metadata/mtg_jamendo_instrument-discogs-effnet-1.json

In [6]:
# Download the model file into the models directory
!wget -q https://essentia.upf.edu/models/feature-extractors/discogs-effnet/discogs-effnet-bs64-1.pb -O models/discogs-effnet-bs64-1.pb

In [1]:
import json
from essentia.standard import MonoLoader, TensorflowPredictEffnetDiscogs, TensorflowPredict2D

class AudioInstrumentClassifier:
    def __init__(self,instrument_model_path,model_json_path,embedding_model_path):

        self.models = []
        self.models_names = []
        self.metadatas = []
        self.instrument_model_path = instrument_model_path
        self.model_json_path = model_json_path
        self.embedding_model_path = embedding_model_path

    #Load Instrument Model and its JSON Metadata.
    def load_instrument_model(self):

        # Load Instrument Prediction Model and its Metadata Path
        model_path = self.instrument_model_path
        metadata_path = self.model_json_path

        # Load metadata
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)

        # Prepare model
        model = TensorflowPredict2D(graphFilename=model_path)

        return model,metadata

    #Load Embeddings of the audio.
    def load_embeddings(self,audio):

      embedding_model_path = self.embedding_model_path
      embedding_model = TensorflowPredictEffnetDiscogs(graphFilename=embedding_model_path, output="PartitionedCall:1")
      embeddings = embedding_model(audio)

      return embeddings

    def predict(self, audio_path, sample_rate=16000):

        # Load audio
        loader = MonoLoader(sampleRate=sample_rate, filename=audio_path)
        audio = loader()

        #Load embeddings from Discog model.
        embeddings = self.load_embeddings(audio)

        #Load Instrument Model and MetaData.
        model,metadata = self.load_instrument_model()

        predictions = model(embeddings)

        #Take Mean across-each timeStamp
        predictions = predictions.mean(axis=0)
        results = {}

        for label, probability in zip(metadata['classes'], predictions):
            results[label] = float(f'{100 * probability:.1f}')

        #Get Top3 Instruments within Music.
        results = self.get_top_3_predictions(results)

        return results

    def get_top_3_predictions(self, predictions):
        """
        Get top 3 predictions from a prediction dictionary

        Args:
            predictions (dict): Predictions dictionary

        Returns:
            list: Top 3 predictions as [(label, probability)]
        """
        # Sort predictions by probability in descending order
        return sorted(predictions.items(), key=lambda x: x[1], reverse=True)[:3]

    def print_predictions(self, predictions):

      print(f"\nInstrument Model Predictions:")
      for label, probability in predictions.items():
          print(f'{label}: {probability}%')


Unable to import NumPy C API from Essentia module. Error code = -1


ModuleNotFoundError: No module named 'numpy.core'

# GENRE CLASSIFICATION

In [8]:
# Download the mtg-jamendo-genre-model file into the models directory
!wget -q https://essentia.upf.edu/models/classification-heads/mtg_jamendo_genre/mtg_jamendo_genre-discogs-effnet-1.pb -O models/mtg_jamendo_genre-discogs-effnet-1.pb
# Download the mtg-jamendo-metadata file into the metadata directory
!wget -q https://essentia.upf.edu/models/classification-heads/mtg_jamendo_genre/mtg_jamendo_genre-discogs-effnet-1.json -O metadata/mtg_jamendo_genre-discogs-effnet-1.json

#Download the msd-musicnn model and metadata.
!wget -q https://essentia.upf.edu/models/autotagging/msd/msd-musicnn-1.pb -O models/msd-musicnn-1.pb
!wget -q https://essentia.upf.edu/models/autotagging/msd/msd-musicnn-1.json -O metadata/msd-musicnn-1.json


In [9]:
import json
import numpy as np
from essentia.standard import MonoLoader, TensorflowPredictEffnetDiscogs, TensorflowPredict2D,TensorflowPredictMusiCNN , TensorflowPredictVGGish

class AudioGenreClassifier:
    def __init__(self,
                 genre_model_path,
                 model_json_path,
                 essentia_genre_model_path,
                 essentia_genre_json_path,
                 embedding_model_path):

        self.models = []
        self.models_names = []
        self.metadatas = []
        #Genre Models From MTG-Jamendro Dataset.
        self.genre_model_path = genre_model_path
        self.model_json_path = model_json_path

        #Genre Model Ussing Essentia AutoTagging.
        self.essentia_genre_model_path = essentia_genre_model_path
        self.essentia_genre_json_path = essentia_genre_json_path

        #Embedding Model for MTG.
        self.embedding_model_path = embedding_model_path

    def load_genre_model(self):

        # Load Genre Prediction Model and its Metadata Path
        model_path = self.genre_model_path
        metadata_path = self.model_json_path

        # Load metadata
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)

        # Prepare model
        model = TensorflowPredict2D(graphFilename=model_path)

        return model,metadata

    def load_essentia_autotagging_model(self):

      with open(self.essentia_genre_json_path, 'r') as f:
          metadata = json.load(f)

      model = TensorflowPredictMusiCNN(graphFilename=self.essentia_genre_model_path)

      return model,metadata

    def load_embeddings(self,audio):

      embedding_model_path = self.embedding_model_path
      embedding_model = TensorflowPredictEffnetDiscogs(graphFilename=embedding_model_path, output="PartitionedCall:1")
      embeddings = embedding_model(audio)

      return embeddings

    def get_top_3_predictions(self, predictions):
        """
        Get top 3 predictions from a prediction dictionary

        Args:
            predictions (dict): Predictions dictionary

        Returns:
            list: Top 3 predictions as [(label, probability)]
        """
        # Sort predictions by probability in descending order
        return sorted(predictions.items(), key=lambda x: x[1], reverse=True)[:3]

    def predict(self, audio_path, sample_rate=16000):

        # Load audio
        loader = MonoLoader(sampleRate=sample_rate, filename=audio_path)
        audio = loader()

        #Load embeddings from Discog model.
        embeddings = self.load_embeddings(audio)

        #Load Genre Model and MetaData.
        model,metadata = self.load_genre_model()

        #Load Essentia Auto-Tagging Model and MetaData.
        essentia_model, essentia_metadata = self.load_essentia_autotagging_model()


        #Take Mean across-each timeStamp for mtg.
        mtg_predictions = model(embeddings)
        mtg_predictions = mtg_predictions.mean(axis=0)
        mtg_results = {}
        for label, probability in zip(metadata['classes'], mtg_predictions):
          mtg_results[label] = float(f'{100 * probability:.1f}')

        #Take Mean across-each timeStamp for essentia.
        essentia_predictions = essentia_model(audio)
        essentia_predictions = essentia_predictions.mean(axis=0)
        essentia_genre_results = {}
        for label, probability in zip(essentia_metadata['classes'], essentia_predictions):
          essentia_genre_results[label] = float(f'{100 * probability:.1f}')

        #Get Top3 Genres Of Music.
        mtg_results = self.get_top_3_predictions(mtg_results)
        essentia_genre_results = self.get_top_3_predictions(essentia_genre_results)

        return mtg_results , essentia_genre_results

    def print_predictions(self, predictions):

      print(f"\Genre Model Predictions:")
      for label, probability in predictions.items():
          print(f'{label}: {probability}%')

# SENTIMENT ANALYSIS AND LYRICS EXTRACTION

In [10]:
!pip install python-dotenv
from dotenv import load_dotenv

In [11]:
from huggingface_hub import login, InferenceClient
# from dotenv import load_dotenv
from google.colab import userdata


import json
import os

load_dotenv(dotenv_path='.env')

#LYRICS BASED SENTIMENT ANALYSIS.

class LyricsExtractor:
    def __init__(self, model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"):
        """
        Initializes the sentiment analyzer with a specified model.

        Args:
            model_name (str): Hugging Face model name for LLM. Defaults to "mistralai/Mixtral-8x7B-Instruct-v0.1".
        """

        #Login to HuggingFace Hub.
        login(token=userdata.get('hf_mixtral_tk'),add_to_git_credential=True)

        # Using Mixtral , because it ranks great at chatbot arena, for generation of QA Pairs.
        repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

        self.llm_client = InferenceClient(
            model=repo_id,
            timeout=200,
        )


    def call_llm(self,
                 inference_client: InferenceClient,
                 prompt: str,
                 max_tokens:int):

        response = inference_client.post(
            json={
                "inputs": prompt,
                "parameters": {"max_new_tokens": max_tokens},
                "task": "text-generation",
            },
        )
        return json.loads(response.decode())[0]["generated_text"]


    def analyze_sentiment(self, lyrics_text, max_new_tokens=60):
        """
        Analyzes the sentiment of the provided lyrics.

        Args:
            lyrics_text (str): Lyrics text to analyze.
            max_new_tokens (int): Maximum number of tokens to generate.

        Returns:
            str: Sentiment analysis generated by the model.
        """

        # Create the prompt for sentiment analysis
        prompt = (
            f"Below are some song lyrics:\n\n{lyrics_text}\n\n"
            "Analyze the text above and describe the overall sentiment and mood of these lyrics in one short and concise sentence. Provide your response after the [RESULT] token.\n\n [RESULT]"
        )

        # Generate the result
        result_text = self.call_llm(self.llm_client, prompt, max_new_tokens)

        # Return the generated text
        return result_text

In [12]:
!pip install -q spleeter
!pip install protobuf==3.20.3
!pip install -q openai-whisper
!pip install webrtcvad

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.10 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
google-cloud-aiplatform 1.73.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
google-cloud-bigquery-connection 1.16.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
google-cloud-bigquery-storage 2.27.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
google-cloud-bigtable 2.27.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,

In [13]:
# SENTIMENT ANALYSIS
import os
import numpy as np
import librosa
import webrtcvad
from spleeter.separator import Separator
import whisper
from transformers import pipeline

class AudioSentimentAnalyzer:
    def __init__(self,
                 LyricsExtractor,
                 model_dir=None):
        """
        Initialize the AudioSentimentAnalyzer.

        Args:
            model_dir (str, optional): Directory to store temporary files
        """
        self.model_dir = model_dir or os.path.join(os.getcwd(), 'audio_analysis_temp')
        self.Lyrics_Analyzer = LyricsExtractor

        # Ensure temporary directory exists
        os.makedirs(self.model_dir, exist_ok=True)

        # Initialize models
        self._init_models()

    def _init_models(self):
        """
        Initialize required models.
        """
        # Vocal Separator
        self.separator = Separator('spleeter:2stems')

        # Whisper for transcription
        self.whisper_model = whisper.load_model("base")

        # Sentiment Analysis Pipeline
        self.sentiment_analyzer = pipeline("sentiment-analysis")

    def check_vocals_presence(self,audio_to_check_vocals,aggressiveness):
        """
        Check if vocals are present in the audio file.

        Returns:
            bool: True if vocals are present, False otherwise
        """

        print(f'Vocals Audio File Path:{audio_to_check_vocals}')
        #Load the Audio File in Librosa and check frequencies that whether it has lyrics or not.
        self.y ,self.sr = librosa.load(audio_to_check_vocals,sr=16000,mono=True)

        # Convert to 16-bit PCM
        audio = (self.y * 32767).astype(np.int16)

        # Initialize VAD
        vad = webrtcvad.Vad(aggressiveness)

        # Frame the audio (30ms frames - recommended for VAD)
        frame_duration = 30  # ms
        frame_size = int(self.sr * (frame_duration / 1000))

        # Count voice frames
        voice_frames = 0
        total_frames = 0

        for start in range(0, len(audio), frame_size):
            frame = audio[start:start+frame_size]

            # Ensure frame is the correct length
            if len(frame) == frame_size:
                total_frames += 1
                try:
                    if vad.is_speech(frame.tobytes(), self.sr):
                        voice_frames += 1
                except Exception as e:
                    print(f"Error processing frame: {e}")
                    continue

        # Calculate vocal presence ratio
        vocal_ratio = voice_frames / total_frames if total_frames > 0 else 0

        return {
            'has_vocals': vocal_ratio > 0.2,
            'vocal_ratio': vocal_ratio,
            'total_frames': total_frames,
            'voice_frames': voice_frames
        }


    def separate_vocals(self,audio_path):
        """
        Separate vocals from the audio file.

        Returns:
            str: Path to extracted vocals file or None
        """
        try:
            # Separate to temporary directory
            output_dir = os.path.join(self.model_dir, 'separated')
            self.separator.separate_to_file(audio_path, output_dir)

            # Find vocals file
            vocals_path = os.path.join(output_dir, os.path.splitext(os.path.basename(audio_path))[0], 'vocals.wav')

            return vocals_path if os.path.exists(vocals_path) else None
        except Exception as e:
            print(f"Error separating vocals: {e}")
            return None

    def transcribe_vocals(self, vocals_path):
        """
        Transcribe vocals to text.

        Args:
            vocals_path (str): Path to vocals audio file

        Returns:
            str: Transcribed lyrics
        """
        try:
            result = self.whisper_model.transcribe(vocals_path)
            return result["text"]
        except Exception as e:
            print(f"Error transcribing vocals: {e}")
            return None

    def analyze_sentiment(self, text):
        """
        Analyze sentiment of given text.

        Args:
            text (str): Text to analyze

        Returns:
            dict: Sentiment analysis results
        """
        try:
            # If text is too short or empty, return None
            if not text or len(text.strip()) < 5:
                return None

            # Use sentiment analysis pipeline
            sentiment_result = self.sentiment_analyzer(text)[0]
            return sentiment_result
        except Exception as e:
            print(f"Error analyzing sentiment: {e}")
            return None

    def analyze(self,audio_path, moods):
        """
        Perform comprehensive audio analysis.

        Args:
            audio_path (str): Path to audio file
            moods (str, optional): Predefined mood if no lyrics found

        Returns:
            dict: Comprehensive analysis results
        """

        # Separate vocals from music first, then check whther the serperated file actually has vocals in it.
        vocals_path = self.separate_vocals(audio_path)

        #Check for Vocals Presence.
        vocals_features = self.check_vocals_presence(vocals_path,aggressiveness=3)

        # Initialize result dictionary
        result = {}
        has_lyrics = False

        if vocals_features['has_vocals']:

          # Attempt lyric transcription
          lyrics = None
          if vocals_path:
              lyrics = self.transcribe_vocals(vocals_path)

              sentiment = self.Lyrics_Analyzer.analyze_sentiment(lyrics)
              result.update({
                  'lyrics': lyrics,
                  'sentiment': sentiment
              })

              has_lyrics = True
        else:
            # If no lyrics, use predefined mood
            mood_prompt = f"Describe the sentiment of a song with following {moods[0]},{moods[1]}, {moods[2]} moods."
            sentiment = self.analyze_sentiment(mood_prompt)
            result.update({
                'lyrics': "No Lyrics, Sentiment Based on Mood",
                'sentiment': f"The Sentiment of the Music is {sentiment['label']}"
            })

        return result, has_lyrics

    def cleanup(self):
        """
        Clean up temporary files and directories.
        """
        try:
            import shutil
            if os.path.exists(self.model_dir):
                shutil.rmtree(self.model_dir)
        except Exception as e:
            print(f"Error during cleanup: {e}")

# Lyrics = LyricsExtractor()
# sentiment_anz = AudioSentimentAnalyzer(LyricsExtractor=Lyrics)
# sentiment_anz.analyze(audio_path='audio_files/A Beacon of Hope.mp3',moods=['happy','sad'])

In [14]:
!pip install protobuf==3.20.3

# Main Function

In [20]:
import os
import csv
import pandas as pd

def process_directory(audio_dir,
                      output_path,
                      genre_classifier,
                      mood_classifier,
                      Instrument_classifier,
                      KeyBPM,
                      Sentiment_analyzer):
    """
    Process audio files, skipping those already processed
    """
    # Ensure output directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Try reading with different encodings
    encodings_to_try = ['latin-1', 'iso-8859-1', 'cp1252']
    processed_files = set()

    for encoding in encodings_to_try:
        try:
            existing_df = pd.read_csv(output_path, encoding=encoding)
            print(f"Successfully read file with {encoding} encoding")
            processed_files = set(existing_df['Filename'])
            break
        except Exception as e:
            print(f"Failed to read with {encoding} encoding: {e}")

    # If CSV doesn't exist, create it with headers
    if not os.path.exists(output_path):
        with open(output_path, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            header = ['Filename',
                      'Essentia-Autotagging-Genre1', 'Genre1_Prob',
                      'Essentia-Autotagging-Genre2', 'Genre2_Prob',
                      'Essentia-Autotagging-Genre3', 'Genre3_Prob',
                      'MTG-Jamendo-Genre1', 'Genre1_Prob',
                      'MTG-Jamendo-Genre2', 'Genre2_Prob',
                      'MTG-Jamendo-Genre3', 'Genre3_Prob',
                      'Essentia-Mood1', 'Mood1_Prob',
                      'Essentia-Mood2', 'Mood2_Prob',
                      'Essentia-Mood3', 'Mood3_Prob',
                      'MTG-Jamendo-Instrument1', 'Instrument1_Prob',
                      'MTG-Jamendo-Instrument2', 'Instrument2_Prob',
                      'MTG-Jamend0-Instrument3', 'Instrument3_Prob',
                      'essentia.Key_edma',
                      'essentia.key_krumhansl',
                      'essentia.key_temperley',
                      'librosa.key',
                      'essentia.rhythm.bpm',
                      'librosa.beat_track (BPM)',
                      'librosa.beat.tempo (BPM)',
                      'Lyrics',
                      'Sentiment Anaylsis']
            csv_writer.writerow(header)

    # Prepare results dictionary
    all_results = {}

    # Process each audio file in the directory
    for filename in os.listdir(audio_dir):
        # Skip if file already processed
        if filename in processed_files:
            print(f"Skipping already processed file: {filename}")
            continue

        if filename.endswith(('.wav', '.mp3', '.flac')):  # Add more extensions if needed
            file_path = os.path.join(audio_dir, filename)

            try:
                # Get Top3 Genre, Mood and Instruments for the Music File
                mtg_genre_predictions , essentia_tagging_predictions  = genre_classifier.predict(file_path)
                print(f'MTG-Genre Predictions for {filename}: {mtg_genre_predictions}')
                print(f'Essentia-Genre Predictions for {filename}: {essentia_tagging_predictions}')


                mood_predictions = mood_classifier.predict(file_path)
                print(f'Mood Predictions for {filename}: {mood_predictions}')

                instrument_predictions = Instrument_classifier.predict(file_path)
                print(f'Instrument Predictions for {filename}: {instrument_predictions}')

                # Get Key and BPM
                key_n_bpm = KeyBPM.analyze(file_path=file_path)

                # Sentiment Analysis
                sentiment_results, has_lyrics = Sentiment_analyzer.analyze(
                    file_path,
                    moods=[mood for mood, prob in mood_predictions[:]]
                )

                # Process sentiment if lyrics exist
                if has_lyrics:
                    result_token = "[RESULT]"
                    sentiment_results['sentiment'] = sentiment_results['sentiment'].split(result_token, 1)[1].strip()
                    sentiment_results['sentiment'] = sentiment_results['sentiment'].split(result_token)[1]

                print(f"Sentiment for {filename}: {sentiment_results['sentiment']}")

                # Prepare row for CSV
                row = [filename]

                # Add MTG-genres
                for genre, prob in mtg_genre_predictions:
                    row.extend([genre, prob])
                # Pad with empty values if less than 3 genres
                while len(row) < 7:
                    row.extend(['', ''])

                # Add Essentia-AutoTagging genres
                for genre, prob in essentia_tagging_predictions:
                    row.extend([genre, prob])
                # Pad with empty values if less than 3 genres
                while len(row) < 13:
                    row.extend(['', ''])

                # Add moods
                for mood, prob in mood_predictions:
                    row.extend([mood, prob])
                # Pad with empty values if less than 3 moods
                while len(row) < 19:
                    row.extend(['', ''])

                # Add instruments
                for instrument, prob in instrument_predictions:
                    row.extend([instrument, prob])
                # Pad with empty values if less than 3 instruments
                while len(row) < 25:
                    row.extend(['', ''])

                # Add keys
                for key in key_n_bpm['Key']:
                    row.extend([key])
                # Pad with empty values if less than 4 keys
                while len(row) < 29:
                    row.append('')

                for bpm in key_n_bpm['BPM']:
                    row.extend([bpm])
                # Pad with empty values if less than 3 BPMs
                while len(row) < 32:
                    row.append('')

                # Add BPM, lyrics, and sentiment
                row.extend([
                    sentiment_results['lyrics'],
                    sentiment_results['sentiment']
                ])

                # Append to CSV immediately after processing each file
                with open(output_path, 'a', newline='') as csvfile:
                    csv_writer = csv.writer(csvfile)
                    csv_writer.writerow(row)

                # Store results (optional)
                all_results[filename] = {
                    'mtg_genres': mtg_genre_predictions,
                    'essentia_genres': essentia_tagging_predictions,
                    'moods': mood_predictions,
                    'instruments': instrument_predictions,
                    'key': key_n_bpm['Key'],
                    'bpm': key_n_bpm['BPM'],
                    'lyrics': sentiment_results['lyrics'],
                    'sentiment': sentiment_results['sentiment']
                }

                print(f"Processed and saved results for {filename}")

            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
                continue

    return all_results


In [17]:
# Load each music analysis Model.

#Create Genre Classifier.
Genreclassifier = AudioGenreClassifier(genre_model_path="models/mtg_jamendo_genre-discogs-effnet-1.pb",
                                        model_json_path="metadata/mtg_jamendo_genre-discogs-effnet-1.json",
                                        essentia_genre_model_path="models/msd-musicnn-1.pb",
                                        essentia_genre_json_path="metadata/msd-musicnn-1.json",
                                        embedding_model_path="models/discogs-effnet-bs64-1.pb")
#Create Mood Classifier.
MoodClassifier = AudioMoodClassifier(models_dir='models/mood_detection_models',
                                  metadatas_dir='metadata/mood_detection_metadatas')

#Create Instrument Classifier
Instrument_classifier = AudioInstrumentClassifier(instrument_model_path="models/mtg_jamendo_instrument-discogs-effnet-1.pb",
                                        model_json_path="metadata/mtg_jamendo_instrument-discogs-effnet-1.json",
                                        embedding_model_path="models/discogs-effnet-bs64-1.pb")
#Create KeyBPMExtractor.
KeyBPM_analyzer = KeyBPMExtractor()

#Create LyricsExtractor Instance.
Lyrics_analyzer = LyricsExtractor()

# Create Sentiment analyzer
Sentiment_analyzer = AudioSentimentAnalyzer(Lyrics_analyzer)


['danceability-musicnn-msd-1.pb', 'mood_aggressive-musicnn-msd-1.pb', 'mood_electronic-musicnn-msd-1.pb', 'mood_happy-musicnn-msd-1.pb', 'mood_party-musicnn-msd-1.pb', 'mood_relaxed-musicnn-msd-1.pb', 'mood_sad-musicnn-msd-1.pb']
['danceability-musicnn-msd-1.json', 'mood_aggressive-musicnn-msd-1.json', 'mood_electronic-musicnn-msd-1.json', 'mood_happy-musicnn-msd-1.json', 'mood_party-musicnn-msd-1.json', 'mood_relaxed-musicnn-msd-1.json', 'mood_sad-musicnn-msd-1.json']
JSON PATH: metadata/mood_detection_metadatas/danceability-musicnn-msd-1.json
JSON PATH: metadata/mood_detection_metadatas/mood_aggressive-musicnn-msd-1.json
JSON PATH: metadata/mood_detection_metadatas/mood_electronic-musicnn-msd-1.json
JSON PATH: metadata/mood_detection_metadatas/mood_happy-musicnn-msd-1.json
JSON PATH: metadata/mood_detection_metadatas/mood_party-musicnn-msd-1.json
JSON PATH: metadata/mood_detection_metadatas/mood_relaxed-musicnn-msd-1.json
JSON PATH: metadata/mood_detection_metadatas/mood_sad-musicnn-

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 153MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more 

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Store results of without vocals in a CSV.
process_directory(audio_dir='/content/drive/MyDrive/Vocals/Vocals', # Give Path to Audio Files (Vocals or Without Vocals.)
                  output_path='/content/drive/MyDrive/vocals_output_v2.csv', #Output CSV Path.
                  genre_classifier=Genreclassifier,
                  mood_classifier=MoodClassifier,
                  Instrument_classifier=Instrument_classifier,
                  KeyBPM=KeyBPM_analyzer,
                  Sentiment_analyzer=Sentiment_analyzer)

#Store Results of With vocals in a CSV.

# process_directory(audio_dir='audio_files',
#                   output_dir='without_vocals',
#                   genre_classifier=Genreclassifier,
#                   mood_classifier=MoodClassifier,
#                   Instrument_classifier=Instrument_classifier,
#                   KeyBPM=KeyBPM_analyzer,
#                   Sentiment_analyzer=Sentiment_analyzer)

Successfully read file with latin-1 encoding
MTG-Genre Predictions for Eve of Enchantment.mp3: [('classical', 24.4), ('pop', 23.9), ('soundtrack', 10.5)]
Essentia-Genre Predictions for Eve of Enchantment.mp3: [('oldies', 24.6), ('country', 16.5), ('easy listening', 12.3)]
Mood Predictions for Eve of Enchantment.mp3: [('sad', 96.2), ('relaxed', 85.0), ('happy', 10.6)]
Instrument Predictions for Eve of Enchantment.mp3: [('piano', 40.6), ('drums', 23.5), ('violin', 16.2)]
Detected Key Scales are: ['D minor', 'D minor', 'D minor', 'D']


<ipython-input-4-9545dabf2dce>:94: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  bpms.append(round(float(tempo)))
<ipython-input-4-9545dabf2dce>:97: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(y=self.y, sr=self.sr)[0]


INFO:spleeter:Downloading model archive https://github.com/deezer/spleeter/releases/download/v1.4.0/2stems.tar.gz


INFO:spleeter:Downloading model archive https://github.com/deezer/spleeter/releases/download/v1.4.0/2stems.tar.gz


INFO:spleeter:Validating archive checksum


...

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Sentiment for Moments in Time.wav:  The overall sentiment and mood of these lyrics convey a sense of nostalgia, longing, and the passage of time.
Processed and saved results for Moments in Time.wav
MTG-Genre Predictions for Under the Christmas Star.mp3: [('classical', 27.4), ('orchestral', 11.5), ('pop', 11.2)]
Essentia-Genre Predictions for Under the Christmas Star.mp3: [('oldies', 38.2), ('jazz', 21.3), ('easy listening', 13.4)]
Mood Predictions for Under the Christmas Star.mp3: [('sad', 70.3), ('happy', 55.5), ('relaxed', 51.3)]
Instrument Predictions for Under the Christmas Star.mp3: [('piano', 40.4), ('drums', 20.3), ('violin', 11.9)]
Detected Key Scales are: ['C minor', 'C minor', 'C minor', 'G']


<ipython-input-4-9545dabf2dce>:94: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  bpms.append(round(float(tempo)))
<ipython-input-4-9545dabf2dce>:97: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(y=self.y, sr=self.sr)[0]


Vocals Audio File Path:/content/audio_analysis_temp/separated/Under the Christmas Star/vocals.wav


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
